In [ ]:
import requests
import json
import sqlite3
import pandas as pd
import tokens.credential

from pathlib import Path
from utils import *

MY_ACCESS_TOKEN = credential.get_token()

In [ ]:
# 일별주가요청
def fn_ka10086(token, data, cont_yn='N', next_key=''):
	# 1. 요청할 API URL
	host = 'https://mockapi.kiwoom.com' # 모의투자
	endpoint = '/api/dostk/mrkcond'
	url =  host + endpoint

	# 2. header 데이터
	headers = {
		'Content-Type': 'application/json;charset=UTF-8', # 컨텐츠타입
		'authorization': f'Bearer {token}', # 접근토큰
		'cont-yn': cont_yn, # 연속조회여부
		'next-key': next_key, # 연속조회키
		'api-id': 'ka10086', # TR명
	}

	# 3. http POST 요청
	response = requests.post(url, headers=headers, json=data)

	# 4. 응답 상태 코드와 데이터 출력
	print('Code:', response.status_code)
	print('Header:', json.dumps({key: response.headers.get(key) for key in ['next-key', 'cont-yn', 'api-id']}, indent=4, ensure_ascii=False))
	return response.json()

In [ ]:
# 2. 요청 데이터
qry_dt = pd.Timestamp.today()
stk_cd = '005930' # 삼성전자

params = {
	'stk_cd': stk_cd, # 종목코드 거래소별 종목코드 (KRX:039490,NXT:039490_NX,SOR:039490_AL)
	'qry_dt': qry_dt.strftime("%Y%m%d"), # 조회일자 YYYYMMDD
	'indc_tp': '0', # 표시구분 0:수량, 1:금액(백만원)
}

records = []
# 3. API 실행
for i in range(2):
    params['qry_dt'] = (qry_dt - pd.offsets.BDay(21*i)).strftime("%Y%m%d")
    data = fn_ka10086(token=MY_ACCESS_TOKEN, data=params)
    for row in data['daly_stkpc']:
        records.append({
        "stk_cd": stk_cd,
        "date": row["date"],
        "open": to_int(row["open_pric"]),
        "high": to_int(row["high_pric"]),
        "low": to_int(row["low_pric"]),
        "close": to_int(row["close_pric"]),
        "volume": to_int(row["trde_qty"]),
    })


In [ ]:
df = pd.DataFrame(records)
df = df.sort_values("date").reset_index(drop=True)

In [ ]:
df.style.format(thousands=',')

In [ ]:
with sqlite3.connect("market.db") as conn:
    conn.execute("""
    CREATE TABLE IF NOT EXISTS daily_price (
        stk_cd  TEXT NOT NULL,
        date    DATE NOT NULL,
        open    INTEGER NOT NULL,
        high    INTEGER NOT NULL,
        low     INTEGER NOT NULL,
        close   INTEGER NOT NULL,
        volume  INTEGER NOT NULL,
        PRIMARY KEY (stk_cd, date)
    )
    """)
    conn.commit()


In [ ]:
with sqlite3.connect("market.db") as conn:
    df.to_sql("tmp_daily_price", conn, if_exists="replace", index=False)

    conn.execute("""
    INSERT OR REPLACE INTO daily_price (stk_cd, "date", open, high, low, close, volume)
    SELECT stk_cd, "date", open, high, low, close, volume
    FROM tmp_daily_price;
    """)

    conn.execute("DROP TABLE tmp_daily_price;")


In [ ]:
# 종목정보 리스트
def fn_ka10099(token, data, cont_yn='N', next_key=''):
	# 1. 요청할 API URL
	host = 'https://mockapi.kiwoom.com' # 모의투자
	endpoint = '/api/dostk/stkinfo'
	url =  host + endpoint

	# 2. header 데이터
	headers = {
		'Content-Type': 'application/json;charset=UTF-8', # 컨텐츠타입
		'authorization': f'Bearer {token}', # 접근토큰
		'cont-yn': cont_yn, # 연속조회여부
		'next-key': next_key, # 연속조회키
		'api-id': 'ka10099', # TR명
	}

	# 3. http POST 요청
	response = requests.post(url, headers=headers, json=data)

	# 4. 응답 상태 코드와 데이터 출력
	print('Code:', response.status_code)
	print('Header:', json.dumps({key: response.headers.get(key) for key in ['next-key', 'cont-yn', 'api-id']}, indent=4, ensure_ascii=False))
	return response.json()["list"]

# 실행 구간
if __name__ == '__main__':

	# 2. 요청 데이터
	params = {
		'mrkt_tp': '0', # 시장구분 0:코스피,10:코스닥,3:ELW,8:ETF,30:K-OTC,50:코넥스,5:신주인수권,4:뮤추얼펀드,6:리츠,9:하이일드
	}

	# 3. API 실행
	data = fn_ka10099(token=MY_ACCESS_TOKEN, data=params)

	# next-key, cont-yn 값이 있을 경우
	# fn_ka10099(token=MY_ACCESS_TOKEN, data=params, cont_yn='Y', next_key='nextkey..')

In [ ]:
df = pd.DataFrame(data)

In [ ]:
df